In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scipy.sparse as sp
import numpy as np
from tensorflow.contrib.rnn import RNNCell
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from sklearn.metrics import mean_squared_error,mean_absolute_error
#import matplotlib.pyplot as plt
import time
time_start=time.time()

In [2]:
# Loading of Dataset-SZ Traffic
def load_sz_data():
    sz_adj = pd.read_csv('sz_adj.csv',header=None)
    adj = np.mat(sz_adj)
    sz_tf = pd.read_csv('sz_speed.csv')
    return sz_tf, adj

data, adj = load_sz_data()

In [3]:
print(data) # Time sequence data

          90217      90218      90219      90220      90221      90222  \
0      8.471971  18.455410  20.590635  15.345258   9.585218  21.501821   
1      7.807137  15.713816  27.523695  11.087895   9.455280  17.332246   
2      8.809457   8.979647  20.280394  16.523419   8.003314  15.789483   
3     51.590372  23.631243  20.224094  15.116459   6.642644  17.575806   
4     58.770433  20.437740  20.465606  14.820217  11.344404   0.000000   
5     58.289126  10.332738  25.331018  18.485616   2.028513  10.718488   
6     61.334449  20.818480  15.839392   8.768217  13.933326  13.866124   
7     58.903144   8.291826  44.043729  15.250859   0.000000   0.000000   
8     57.210441  25.765667  18.677565   5.066051   0.000000  12.645903   
9     59.559265  16.803983  20.314610  23.393083   2.612821   0.000000   
10    60.217867  26.773576  17.684078  17.735375  12.957171   7.404007   
11    56.336405  18.866552  27.502295  18.201105  23.973238  22.215358   
12    58.511395  15.802270  15.900551 

In [4]:
print(adj) #Adjacency Matrix of Data

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 1 0]]


In [5]:
print(data.shape)
time_len=data.shape[0] # Time sequence length
num_nodes=data.shape[1] #Number of Roads

(2976, 156)


In [6]:
# Setting the variables
output_dim=pre_len=1
seq_len=4
num_units=100
train_rate=0.8
batch_size=32

In [7]:
#Normalization : Traffic Speed Data

data1 =np.mat(data,dtype=np.float32)

max_value = np.max(data1)
data1  = data1/max_value
print(data1)

   

[[0.0980221  0.21353212 0.23823702 ... 0.19194394 0.37707424 0.        ]
 [0.09032986 0.18181142 0.31845367 ... 0.45834997 0.37705195 0.        ]
 [0.10192686 0.10389599 0.23464748 ... 0.3856561  0.43354756 0.        ]
 ...
 [0.37947276 0.141638   0.10453826 ... 0.13605069 0.1998609  0.        ]
 [0.39722532 0.18600048 0.121989   ... 0.16306393 0.16957766 0.        ]
 [0.38101357 0.14001252 0.10420388 ... 0.19054307 0.1845446  0.        ]]


In [8]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate) #2976 *0.8 =2380
    train_data = data[0:train_size] #  [0:2380]
    test_data = data[train_size:time_len] #[2380:2976]
    print(train_data.shape,'----->',test_data.shape)

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len): #(2380-4-1)=2375
        a = train_data[i: i + seq_len + pre_len] #[0:0+4+1] 
        trainX.append(a[0 : seq_len]) #a[0:4] 4 time * 156 roads
        trainY.append(a[seq_len : seq_len + pre_len]) #a[4:4+1] 1 time*156 
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
      
    trainX1 = np.array(trainX) 
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)
    testY1 = np.array(testY)
    return trainX1, trainY1, testX1, testY1

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)  
print('Train Test Split Details :')
print('Train x ----> ',len(trainX))
print('Train y ----> ',len(trainY))
print(trainX.shape)
print('Test x ----> ',len(testX))
print('Test y ----> ',len(testY))
#print('\nTrain Sample Details :')
#print(trainX[0],'--->',trainY[0])


(2380, 156) -----> (596, 156)
Train Test Split Details :
Train x ---->  2375
Train y ---->  2375
(2375, 4, 156)
Test x ---->  591
Test y ---->  591


In [9]:
def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name)

In [10]:
class stgcnCell(RNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(stgcnCell, self).__init__(_reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(self.calculate_laplacian(adj))

    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    def calculate_laplacian(self,adj, lambda_max=1):  
        adj = self.normalized_adj(adj + sp.eye(adj.shape[0])) # normalisation(self identity matrix + adj)
        adj = sp.csr_matrix(adj) #compressed sparse matrix
        adj = adj.astype(np.float32)
        return self.sparse_to_tuple(adj)
    
    def normalized_adj(self,adj):
        adj = sp.coo_matrix(adj)
        degree = np.array(adj.sum(1)) # Degree Matrix row wise sum
        d_inv_sqrt = np.power(degree, -0.5).flatten() # D inv = Degree ^-0.5 
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt) #substitution of the 1D array degree in a 2D matrix diagonals
        normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # norm= D^-0.5 * adj * D^-0.5
        normalized_adj = normalized_adj.astype(np.float32) 
        return normalized_adj
    
    def sparse_to_tuple(self,mx):
        mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() #coordinate stacking row and column wise and transpose
        L = tf.SparseTensor(coords, mx.data, mx.shape) # mx.shape= (156,156)
        #print('shape ---->',mx.shape)
        return tf.sparse_reorder(L) #row major ordering
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.variable_scope(scope or "tgcn"):
            with tf.variable_scope("gates"):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope)) #ut (or) rt = sigma(Wu [f(A;Xt); h{t-1}] + bu)
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.variable_scope("candidate"):
                r_state = r * state #r* h{t-1}
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))#ct = tanh(Wc [f(A;Xt); r_state] + bc) 
            new_h = u * state + (1 - u) * c #ht = ut * h{t-1} + (1 - u{t}) * ct
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)#None,156,None
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units)) #32,156,64
        ## concat
        x_s = tf.concat([inputs, state], axis=2) #32,156,65
        input_size = x_s.get_shape()[2].value #65
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0]) #156,65,32
        x0 = tf.reshape(x0, shape=[self._nodes, -1]) #156,65*32
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for m in self._adj:#1,156
                x1 = tf.sparse_tensor_dense_matmul(m, x0) #1,65*32
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1]) #156,65,32
            x = tf.transpose(x,perm=[2,0,1]) #32,156,65
            x = tf.reshape(x, shape=[-1, input_size]) #156*32,65
            weights = tf.get_variable( # 65,64
                'weights', [input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size) 
            biases = tf.get_variable( #64
                "biases", [output_size], initializer=tf.constant_initializer(bias, dtype=tf.float32))
            x = tf.nn.bias_add(x, biases) #biases added
            x = tf.reshape(x, shape=[-1, self._nodes, output_size]) #32,156,64
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])#32,156*64
        return x

In [11]:
#inputs = tf.convert_to_tensor(trainX, np.float32)
#labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
#g=stgcnCell(num_units, adj, inputs, output_dim)

In [12]:
def STGCN(_X, _weights, _biases):
    ###
    cell_1 = stgcnCell(num_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True) #stack rnn cells
    _X = tf.unstack(_X, axis=1) # 4 tensorflow arrays of shape None,156 (seq_len=4)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32) #Creates a recurrent neural network specified by RNNCell cell
    #4 ouputs and 1 state None,9984(156*64)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,num_units])#None,156,64
        o = tf.reshape(o,shape=[-1,num_units])#None*156,64
        m.append(o) #4 objects
    last_output = m[-1] #last one
    output = tf.matmul(last_output, _weights['out']) + _biases['out'] #multiply with weights and add bias None*156,1+len(1)=156,1
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len]) # None,156,1
    output = tf.transpose(output, perm=[0,2,1])#None,1,156
    output = tf.reshape(output, shape=[-1,num_nodes]) #None*1,156
    return output, m, states

In [13]:
inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
weights = {
    'out': tf.Variable(tf.random_normal([num_units, pre_len], mean=1.0), name='weight_o')} #64,1
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')} #1
print(type(inputs))
pred,ttts,ttto = STGCN(inputs, weights, biases)

Instructions for updating:
Colocations handled automatically by placer.
<class 'tensorflow.python.framework.ops.Tensor'>
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [14]:
y_pred = pred

In [15]:
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])

In [16]:
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)

In [17]:
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))

In [18]:
lr=0.001
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [19]:
variables = tf.global_variables()
training_epoch=1000
saver = tf.train.Saver(tf.global_variables()) #
#sess = tf.Session()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())
out = 'out/%s'%("STGCN")
#out = 'out/%s_%s'%(model_name,'perturbation')
path1 = '%s_%s_lr%r_batch%r_unit%r_seq%r_pre%r_epoch%r'%("STGCN","sz",lr,batch_size,num_units,seq_len,pre_len,training_epoch)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)

In [20]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var

In [21]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]

In [22]:
def acc(a,b):  
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    train_acc=1-F_norm
    return train_acc

In [23]:
for epoch in range(training_epoch):
    print("Epoch ", epoch)
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)
        train_label=np.reshape(mini_label,[-1,num_nodes])
     #print(mini_label.shape,train_output.shape) (32, 1, 156) (32, 156)
     # Test completely at every epoch
    print("Accuracy ----> ", evaluation(train_label,train_output)[2])
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})
    #train_label=np.reshape(trainY,[-1,num_nodes])
    #train_acc=acc(train_label,train_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, test_output)
    test_label1 = test_label * max_value#Inverse normalization
    test_output1 = test_output * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    #print(mini_label.shape,train_output.shape)
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    if (epoch % 500 == 0):        
        saver.save(sess, path+'/model_10015TGCN_pre_%r'%epoch, global_step = epoch)
        
time_end = time.time()
print('Time taken : ',time_end-time_start,'s')

Epoch  0
Accuracy ---->  0.5859107971191406
Iter:0 train_rmse:5.976 test_loss:250.9 test_rmse:0.0737 test_acc:0.5563
Epoch  1
Accuracy ---->  0.6035177111625671
Iter:1 train_rmse:5.722 test_loss:227.8 test_rmse:0.07021 test_acc:0.5773
Epoch  2
Accuracy ---->  0.6091001033782959
Iter:2 train_rmse:5.641 test_loss:221.7 test_rmse:0.06927 test_acc:0.583
Epoch  3
Accuracy ---->  0.6127183437347412
Iter:3 train_rmse:5.589 test_loss:217.8 test_rmse:0.06866 test_acc:0.5867
Epoch  4
Accuracy ---->  0.614863932132721
Iter:4 train_rmse:5.558 test_loss:215.5 test_rmse:0.06829 test_acc:0.5889
Epoch  5
Accuracy ---->  0.6160442531108856
Iter:5 train_rmse:5.541 test_loss:214.2 test_rmse:0.06808 test_acc:0.5901
Epoch  6
Accuracy ---->  0.6166784465312958
Iter:6 train_rmse:5.532 test_loss:213.5 test_rmse:0.06797 test_acc:0.5908
Epoch  7
Accuracy ---->  0.6170701682567596
Iter:7 train_rmse:5.526 test_loss:213.0 test_rmse:0.0679 test_acc:0.5912
Epoch  8
Accuracy ---->  0.6174040138721466
Iter:8 train_rms

Accuracy ---->  0.6449323296546936
Iter:69 train_rmse:5.124 test_loss:190.7 test_rmse:0.06422 test_acc:0.6134
Epoch  70
Accuracy ---->  0.6452835500240326
Iter:70 train_rmse:5.119 test_loss:190.4 test_rmse:0.06417 test_acc:0.6136
Epoch  71
Accuracy ---->  0.6456573605537415
Iter:71 train_rmse:5.113 test_loss:190.1 test_rmse:0.06413 test_acc:0.6139
Epoch  72
Accuracy ---->  0.6460505127906799
Iter:72 train_rmse:5.108 test_loss:189.8 test_rmse:0.06408 test_acc:0.6142
Epoch  73
Accuracy ---->  0.6464574933052063
Iter:73 train_rmse:5.102 test_loss:189.5 test_rmse:0.06402 test_acc:0.6146
Epoch  74
Accuracy ---->  0.6468730270862579
Iter:74 train_rmse:5.096 test_loss:189.2 test_rmse:0.06396 test_acc:0.6149
Epoch  75
Accuracy ---->  0.6472928524017334
Iter:75 train_rmse:5.09 test_loss:188.8 test_rmse:0.0639 test_acc:0.6153
Epoch  76
Accuracy ---->  0.6477153599262238
Iter:76 train_rmse:5.084 test_loss:188.4 test_rmse:0.06384 test_acc:0.6157
Epoch  77
Accuracy ---->  0.6481410264968872
Iter:77

Accuracy ---->  0.6750093400478363
Iter:137 train_rmse:4.69 test_loss:163.5 test_rmse:0.05943 test_acc:0.6422
Epoch  138
Accuracy ---->  0.6753245890140533
Iter:138 train_rmse:4.685 test_loss:163.3 test_rmse:0.05939 test_acc:0.6424
Epoch  139
Accuracy ---->  0.6756345629692078
Iter:139 train_rmse:4.681 test_loss:163.0 test_rmse:0.05935 test_acc:0.6427
Epoch  140
Accuracy ---->  0.6759397089481354
Iter:140 train_rmse:4.676 test_loss:162.8 test_rmse:0.05931 test_acc:0.6429
Epoch  141
Accuracy ---->  0.6762398779392242
Iter:141 train_rmse:4.672 test_loss:162.6 test_rmse:0.05927 test_acc:0.6432
Epoch  142
Accuracy ---->  0.6765362322330475
Iter:142 train_rmse:4.668 test_loss:162.4 test_rmse:0.05923 test_acc:0.6434
Epoch  143
Accuracy ---->  0.6768288016319275
Iter:143 train_rmse:4.664 test_loss:162.2 test_rmse:0.05919 test_acc:0.6437
Epoch  144
Accuracy ---->  0.6771181523799896
Iter:144 train_rmse:4.659 test_loss:161.9 test_rmse:0.05915 test_acc:0.6439
Epoch  145
Accuracy ---->  0.6774052

Accuracy ---->  0.6906516849994659
Iter:205 train_rmse:4.464 test_loss:149.5 test_rmse:0.05681 test_acc:0.658
Epoch  206
Accuracy ---->  0.6908357739448547
Iter:206 train_rmse:4.461 test_loss:149.4 test_rmse:0.05679 test_acc:0.6581
Epoch  207
Accuracy ---->  0.6910208761692047
Iter:207 train_rmse:4.459 test_loss:149.3 test_rmse:0.05677 test_acc:0.6582
Epoch  208
Accuracy ---->  0.6912059485912323
Iter:208 train_rmse:4.456 test_loss:149.2 test_rmse:0.05675 test_acc:0.6584
Epoch  209
Accuracy ---->  0.6913906335830688
Iter:209 train_rmse:4.453 test_loss:149.1 test_rmse:0.05673 test_acc:0.6585
Epoch  210
Accuracy ---->  0.6915735006332397
Iter:210 train_rmse:4.451 test_loss:149.0 test_rmse:0.05671 test_acc:0.6586
Epoch  211
Accuracy ---->  0.6917541921138763
Iter:211 train_rmse:4.448 test_loss:148.9 test_rmse:0.05669 test_acc:0.6587
Epoch  212
Accuracy ---->  0.6919315159320831
Iter:212 train_rmse:4.446 test_loss:148.8 test_rmse:0.05667 test_acc:0.6588
Epoch  213
Accuracy ---->  0.6921045

Accuracy ---->  0.6982159912586212
Iter:273 train_rmse:4.355 test_loss:141.7 test_rmse:0.05531 test_acc:0.667
Epoch  274
Accuracy ---->  0.6983446180820465
Iter:274 train_rmse:4.353 test_loss:141.6 test_rmse:0.05529 test_acc:0.6671
Epoch  275
Accuracy ---->  0.6984750032424927
Iter:275 train_rmse:4.351 test_loss:141.5 test_rmse:0.05526 test_acc:0.6673
Epoch  276
Accuracy ---->  0.6986069679260254
Iter:276 train_rmse:4.349 test_loss:141.4 test_rmse:0.05524 test_acc:0.6674
Epoch  277
Accuracy ---->  0.6987402141094208
Iter:277 train_rmse:4.347 test_loss:141.3 test_rmse:0.05522 test_acc:0.6675
Epoch  278
Accuracy ---->  0.6988749206066132
Iter:278 train_rmse:4.345 test_loss:141.2 test_rmse:0.0552 test_acc:0.6677
Epoch  279
Accuracy ---->  0.6990109086036682
Iter:279 train_rmse:4.343 test_loss:141.1 test_rmse:0.05518 test_acc:0.6678
Epoch  280
Accuracy ---->  0.699147492647171
Iter:280 train_rmse:4.342 test_loss:140.9 test_rmse:0.05515 test_acc:0.668
Epoch  281
Accuracy ---->  0.6992850005

Accuracy ---->  0.7061276137828827
Iter:341 train_rmse:4.241 test_loss:133.9 test_rmse:0.05375 test_acc:0.6764
Epoch  342
Accuracy ---->  0.7062309980392456
Iter:342 train_rmse:4.239 test_loss:133.8 test_rmse:0.05373 test_acc:0.6765
Epoch  343
Accuracy ---->  0.7063346803188324
Iter:343 train_rmse:4.238 test_loss:133.7 test_rmse:0.05371 test_acc:0.6767
Epoch  344
Accuracy ---->  0.706438422203064
Iter:344 train_rmse:4.236 test_loss:133.6 test_rmse:0.05368 test_acc:0.6768
Epoch  345
Accuracy ---->  0.7065428793430328
Iter:345 train_rmse:4.235 test_loss:133.5 test_rmse:0.05366 test_acc:0.6769
Epoch  346
Accuracy ---->  0.7066478729248047
Iter:346 train_rmse:4.233 test_loss:133.4 test_rmse:0.05364 test_acc:0.6771
Epoch  347
Accuracy ---->  0.7067538499832153
Iter:347 train_rmse:4.232 test_loss:133.2 test_rmse:0.05362 test_acc:0.6772
Epoch  348
Accuracy ---->  0.706860363483429
Iter:348 train_rmse:4.23 test_loss:133.1 test_rmse:0.05359 test_acc:0.6773
Epoch  349
Accuracy ---->  0.706968605

Accuracy ---->  0.713787168264389
Iter:409 train_rmse:4.13 test_loss:126.9 test_rmse:0.05231 test_acc:0.6851
Epoch  410
Accuracy ---->  0.7138969302177429
Iter:410 train_rmse:4.129 test_loss:126.8 test_rmse:0.05229 test_acc:0.6852
Epoch  411
Accuracy ---->  0.7140066027641296
Iter:411 train_rmse:4.127 test_loss:126.7 test_rmse:0.05227 test_acc:0.6853
Epoch  412
Accuracy ---->  0.7141149640083313
Iter:412 train_rmse:4.126 test_loss:126.6 test_rmse:0.05225 test_acc:0.6855
Epoch  413
Accuracy ---->  0.7142232358455658
Iter:413 train_rmse:4.124 test_loss:126.5 test_rmse:0.05223 test_acc:0.6856
Epoch  414
Accuracy ---->  0.7143313884735107
Iter:414 train_rmse:4.122 test_loss:126.4 test_rmse:0.05221 test_acc:0.6857
Epoch  415
Accuracy ---->  0.7144386172294617
Iter:415 train_rmse:4.121 test_loss:126.3 test_rmse:0.05219 test_acc:0.6858
Epoch  416
Accuracy ---->  0.7145452797412872
Iter:416 train_rmse:4.119 test_loss:126.2 test_rmse:0.05216 test_acc:0.6859
Epoch  417
Accuracy ---->  0.71465206

KeyboardInterrupt: 

##### 

In [ ]:
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)
var.to_csv(path+'/test_result15.csv',index = False,header = False)
#plot_result(test_result,test_label1,path)
#plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)

In [ ]:
print("Testing:")
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

In [ ]:
print("Training:")
rmse, mae, acc, r2_score, var_score = evaluation(train_label,train_output)
print('min_rmse:%r'%(rmse),
      'min_mae:%r'%(mae),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

In [ ]:
#inverse normalisation
print('min_rmse:%r'%(rmse*max_value),
      'min_mae:%r'%(mae*max_value),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

In [ ]:
import matplotlib.pyplot as plt

def plot_result(test_result,test_label1,path):
    ##all test result visualization
    fig1 = plt.figure(figsize=(7,1.5))
#    ax1 = fig1.add_subplot(1,1,1)
    a_pred = test_result[:,0]
    a_true = test_label1[:,0]
    plt.plot(a_pred,'r-',label='prediction')
    plt.plot(a_true,'b-',label='true')
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/test_all.jpg')
    plt.show()
    ## oneday test result visualization
    fig1 = plt.figure(figsize=(7,1.5))
#    ax1 = fig1.add_subplot(1,1,1)
    a_pred = test_result[0:96,0]
    a_true = test_label1[0:96,0]
    plt.plot(a_pred,'r-',label="prediction")
    plt.plot(a_true,'b-',label="true")
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/test_oneday.jpg')
    plt.show()
    
def plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path):
    ###train_rmse & test_rmse 
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(train_rmse, 'r-', label="train_rmse")
    plt.plot(test_rmse, 'b-', label="test_rmse")
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/rmse.jpg')
    plt.show()
    #### train_loss & train_rmse
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(train_loss,'b-', label='train_loss')
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/train_loss.jpg')
    plt.show()

    fig1 = plt.figure(figsize=(5,3))
    plt.plot(train_rmse,'b-', label='train_rmse')
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/train_rmse.jpg')
    plt.show()

    ### accuracy
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(test_acc, 'b-', label="test_acc")
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/test_acc.jpg')
    plt.show()
    ### rmse
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(test_rmse, 'b-', label="test_rmse")
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/test_rmse.jpg')
    plt.show()
    ### mae
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(test_mae, 'b-', label="test_mae")
    plt.legend(loc='best',fontsize=10)
    plt.savefig(path+'/test_mae.jpg')
    plt.show()

In [ ]:
plot_result(test_result,test_label1,path)
plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)